**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-om9xvk32
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-om9xvk32
  fatal: unable to access 'https://github.com/Kaggle/learntools.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-om9xvk32 did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-om9xvk32 did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,f37cef48d983edb5ae10ed2cdbdea6648cec2b90,ad4b1730fcbfdb84e41313179a688924012db322823f48...,2023-06-01 08:45:00+00:00,2023-06-01 08:45:00+00:00,57,0.0,NaN,NaN,NaN,NaN,...,0.0,3.50,Cash,Flash Cab,NaN,NaN,None,NaN,NaN,None
1,f4eb3ed7cf1456df3aee3d8375100afa2d30866a,b15d5dba7f578c2e93e08c66d7a9b57bc35c9f316265cb...,2023-06-25 19:45:00+00:00,2023-06-25 19:45:00+00:00,8,0.0,NaN,NaN,NaN,NaN,...,0.0,33.00,Credit Card,Taxicab Insurance Agency Llc,NaN,NaN,None,NaN,NaN,None
2,f3c8d1c292d3a4bfcd3c52103d8e9d1b278c4632,c9867d006415cbc16529555f98cdeb44cb53aeaf1d9ae7...,2023-06-03 22:45:00+00:00,2023-06-03 22:45:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,10.25,Cash,Taxi Affiliation Services,NaN,NaN,None,NaN,NaN,None
3,f5105921ebf789606e80d1fb1d736ba2a4093fd7,1ecd824aa3fbb5a17d617c3ce214040c91599cb342f7a7...,2023-06-17 16:45:00+00:00,2023-06-17 17:15:00+00:00,2100,0.0,NaN,NaN,NaN,NaN,...,23.5,50.00,Credit Card,Chicago City Taxi Association,NaN,NaN,None,NaN,NaN,None
4,f3f06434e7a81119d27954adddce9cdb88f927f5,520ceeda508661c3d09a4d7b85c955a6c971d0cca9c109...,2023-06-13 18:15:00+00:00,2023-06-13 18:15:00+00:00,25,0.0,NaN,NaN,NaN,NaN,...,0.0,30.50,Credit Card,Flash Cab,NaN,NaN,None,NaN,NaN,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                             AVG(num_trips)
                                 OVER(
                                     ORDER BY trip_date
                                     ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                                     ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-02-14,92247.142857
1,2016-01-31,80539.000000
2,2016-03-10,98769.714286
3,2016-02-01,80464.571429
4,2016-03-14,102758.000000


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
#q_1.hint()
# q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below
trip_number_query = """
                    SELECT  pickup_community_area,
                            trip_start_timestamp,
                            trip_end_timestamp,
                            RANK() OVER (
                                         PARTITION BY pickup_community_area
                                         ORDER BY trip_start_timestamp
                                        ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,19.0,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,1
1,19.0,2013-10-03 00:00:00+00:00,2013-10-03 00:00:00+00:00,1
2,19.0,2013-10-03 03:30:00+00:00,2013-10-03 04:15:00+00:00,3
3,19.0,2013-10-03 06:45:00+00:00,2013-10-03 06:45:00+00:00,4
4,19.0,2013-10-03 08:00:00+00:00,2013-10-03 08:45:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
#q_2.hint()
# q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(trip_start_timestamp,
                                      LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp),
                                      MINUTE) AS prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,1deedd2ca018f2b78a3d8cb14905a763b5a77b16097fdb...,2013-10-03 19:00:00+00:00,2013-10-03 19:15:00+00:00,390.0
1,29b6866f97b4ee801bebc5fc21758f7092c13f43884c58...,2013-10-03 18:45:00+00:00,2013-10-03 18:45:00+00:00,480.0
2,382c449a49e179aada54d185943d41afc724eae1fca80e...,2013-10-03 06:00:00+00:00,2013-10-03 06:00:00+00:00,255.0
3,45cf200bf4881cb3322eb567ea859d2c81390d8d1fb6c7...,2013-10-03 08:45:00+00:00,2013-10-03 09:15:00+00:00,420.0
4,46182a244f8a81ba3c58545e7259ffe8ce0ade1911da25...,2013-10-03 10:45:00+00:00,2013-10-03 11:00:00+00:00,240.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
#q_3.hint()
# q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*